In [1]:
!pip install xlsxwriter pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.0 MB/s eta 0:00:00
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27433 sha256=2f95919a14b828354a492420e33a8190b326038faa177fbea3332af82fa3e0bc
  Stored in directory: /root/.cache/pip/wheels/6c/10/da/a5b513f5b3916fc391c20ee7b4633e5cf3396d570cdd74970f
Successfully built pydrive


In [2]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
from time import mktime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
import xlsxwriter


#%% functions - data conversion

def date_change_format(date_string):
    """
    Konwertuje datę z różnych formatów na "YYYY-MM-DD"
    """
    try:
        date_string = ' '.join(date_string.strip().split())

        if re.match(r'\d{4}-\d{2}-\d{2}', date_string):
            return date_string[:10]

        if 'T' in date_string:
            return date_string.split('T')[0]

        # Polski słownik miesięcy
        lookup_table = {
            "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
            "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
            "września": "09", "października": "10", "listopada": "11", "grudnia": "12",
            "styczeń": "01", "luty": "02", "marzec": "03", "kwiecień": "04",
            "maj": "05", "czerwiec": "06", "lipiec": "07", "sierpień": "08",
            "wrzesień": "09", "październik": "10", "listopad": "11", "grudzień": "12"
        }

        for k, v in lookup_table.items():
            date_string = date_string.replace(k, v)

        if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
        else:
            result = time.strptime(date_string, "%d %m %Y")

        changed_date = datetime.fromtimestamp(mktime(result))
        return format(changed_date.date())
    except Exception as e:
        return "no date"


#%% functions - link extraction

def get_all_article_links():
    """
    Pobiera wszystkie linki do artykułów ze wszystkich kategorii
    """
    base_url = "https://contekstualni.pl"

    # Kategorie z menu
    categories = {
        'Literatura': '/literatura/',
        'Felietony': '/felietony/',
        'Recenzje': '/recenzje/',
        'Rozmowy': '/rozmowy/',
        'Sztuka': '/sztuka/'
    }

    print("="*60)
    print("KROK 1: Pobieranie artykułów z kategorii")
    print("="*60 + "\n")

    all_articles = []

    for category_name, category_url in categories.items():
        print(f"\nKategoria: {category_name}")
        print("-" * 40)

        # Najpierw pobierz stronę kategorii, żeby sprawdzić paginację
        try:
            r = requests.get(base_url + category_url, timeout=10)
            r.encoding = 'utf-8'

            if r.status_code != 200:
                print(f"  ✗ Błąd {r.status_code}")
                continue

            soup = BeautifulSoup(r.text, 'lxml')

            # Pobierz artykuły ze strony kategorii
            # Artykuły są w <ul class="wp-block-latest-posts__list">
            articles_list = soup.find('ul', class_='wp-block-latest-posts__list')

            if articles_list:
                article_items = articles_list.find_all('li')

                for item in article_items:
                    link = item.find('a', class_='wp-block-latest-posts__post-title')
                    if link and link.get('href'):
                        article_url = link['href']
                        article_title = link.get_text(strip=True)

                        # Excerpt
                        excerpt_div = item.find('div', class_='wp-block-latest-posts__post-excerpt')
                        excerpt = excerpt_div.get_text(strip=True) if excerpt_div else ""

                        all_articles.append({
                            'article_url': article_url,
                            'article_title': article_title,
                            'category': category_name,
                            'excerpt': excerpt
                        })

                print(f"  ✓ Znaleziono {len(article_items)} artykułów")

        except Exception as e:
            print(f"  ✗ Błąd: {e}")

        time.sleep(0.5)

    # Dodatkowo: pobierz ze strony głównej (masonry layout)
    print(f"\n{'='*60}")
    print("Pobieranie ze strony głównej")
    print("="*60)

    try:
        r = requests.get(base_url, timeout=10)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'lxml')

        # Artykuły na stronie głównej: <article class="post-XXX ...">
        articles = soup.find_all('article', class_=lambda x: x and 'post-' in str(x))

        for article in articles:
            # Link do artykułu: <h2 class="entry-title"> <a>
            title_elem = article.find('h2', class_='entry-title')
            if not title_elem:
                continue

            link = title_elem.find('a')
            if not link or not link.get('href'):
                continue

            article_url = link['href']
            article_title = link.get_text(strip=True)

            # Kategoria: <div class="blog-category-list"> <a>
            category = "Uncategorized"
            category_div = article.find('div', class_='blog-category-list')
            if category_div:
                category_link = category_div.find('a')
                if category_link:
                    category = category_link.get_text(strip=True)

            # Excerpt: <div class="entry-content"> <p>
            excerpt = ""
            content_div = article.find('div', class_='entry-content')
            if content_div:
                p = content_div.find('p')
                if p:
                    excerpt = p.get_text(strip=True)

            # Sprawdź, czy już mamy ten artykuł
            if not any(a['article_url'] == article_url for a in all_articles):
                all_articles.append({
                    'article_url': article_url,
                    'article_title': article_title,
                    'category': category,
                    'excerpt': excerpt
                })

        print(f"  ✓ Znaleziono {len(articles)} artykułów")

    except Exception as e:
        print(f"  ✗ Błąd: {e}")

    # Usuń duplikaty
    unique_articles = []
    seen_urls = set()
    for article in all_articles:
        if article['article_url'] not in seen_urls:
            unique_articles.append(article)
            seen_urls.add(article['article_url'])

    print(f"\n{'='*60}")
    print(f"Łącznie znaleziono: {len(unique_articles)} unikalnych artykułów")
    print(f"{'='*60}\n")

    return unique_articles


#%% functions - article scraping

def scrape_article(article_data):
    """
    Pobiera szczegóły pojedynczego artykułu
    """
    try:
        article_url = article_data['article_url']

        r = requests.get(article_url, timeout=15)
        r.encoding = 'utf-8'

        if r.status_code != 200:
            errors.append(article_url)
            return

        soup = BeautifulSoup(r.text, 'lxml')

        # Kategoria: <div class="category-list"> <a>
        category = article_data.get('category', 'no category')
        category_div = soup.find('div', class_='category-list')
        if category_div:
            category_link = category_div.find('a')
            if category_link:
                category = category_link.get_text(strip=True)

        # Tytuł: <h1 class="entry-title">
        title = "no title"
        title_elem = soup.find('h1', class_='entry-title')
        if title_elem:
            title = title_elem.get_text(strip=True)

        # Autor: <span class="author vcard"> <a>
        author = "no author"
        author_span = soup.find('span', class_='author')
        if author_span:
            author_link = author_span.find('a')
            if author_link:
                author = author_link.get_text(strip=True)

        # Data publikacji: <time class="entry-date">
        date_of_publication = "no date"
        time_elem = soup.find('time', class_='entry-date')
        if time_elem and time_elem.get('datetime'):
            date_of_publication = date_change_format(time_elem['datetime'])

        # Treść artykułu: <div class="entry-content">
        text = "no text"
        content_div = soup.find('div', class_='entry-content')
        if content_div:
            paragraphs = []

            # Zbieramy wszystkie paragrafy
            for p in content_div.find_all('p'):
                p_text = p.get_text(strip=True)
                if p_text:
                    paragraphs.append(p_text)

            if paragraphs:
                text = ' '.join(paragraphs)
                text = text.replace('\n', ' ').replace('\xa0', ' ')
                text = re.sub(r'\s+', ' ', text).strip()

        # Linki zewnętrzne
        external_links = []
        if content_div:
            links = content_div.find_all('a', href=True)
            for link in links:
                href = link['href']
                if href.startswith('http') and 'contekstualni.pl' not in href:
                    external_links.append(href)

        external_links_str = ' | '.join(external_links) if external_links else None

        # Zdjęcie wyróżniające: <div class="single-featured-image"> <img>
        images = []
        featured_div = soup.find('div', class_='single-featured-image')
        if featured_div:
            img = featured_div.find('img')
            if img and img.get('src'):
                images.append(img['src'])

        # Inne zdjęcia w treści
        if content_div:
            for img in content_div.find_all('img'):
                if img.get('src'):
                    img_url = img['src']
                    if img_url not in images:
                        images.append(img_url)

        has_images = len(images) > 0
        photos_links = ' | '.join(images) if images else None

        # Filmy - sprawdzamy iframe'y
        has_videos = False
        if content_div:
            iframes = content_div.find_all('iframe')
            has_videos = len(iframes) > 0

        # Tagi: <span class="tags-links"> <a>
        tags = []
        tags_span = soup.find('span', class_='tags-links')
        if tags_span:
            tag_links = tags_span.find_all('a')
            for tag_link in tag_links:
                tag_text = tag_link.get_text(strip=True)
                if tag_text:
                    tags.append(tag_text)

        tags_str = ', '.join(tags) if tags else None

        result = {
            "Link": article_url,
            "Data publikacji": date_of_publication,
            "Kategoria": category,
            "Tytuł artykułu": title,
            "Autor": author,
            "Tekst artykułu": text,
            "Tagi": tags_str,
            "Linki zewnętrzne": external_links_str,
            "Zdjęcia/Grafika": has_images,
            "Filmy": has_videos,
            "Linki do zdjęć": photos_links
        }

        all_results.append(result)

    except Exception as e:
        errors.append(article_data['article_url'])
        print(f"✗ Błąd: {e}")


#%% main execution

if __name__ == "__main__":
    print("\n" + "="*60)
    print("SCRAPER CONTEKSTUALNI.PL")
    print("Blog literacko-kulturalny")
    print("="*60 + "\n")

    # KROK 1: Pobierz linki do artykułów
    article_links = get_all_article_links()

    if not article_links:
        print("Nie znaleziono artykułów!")
        exit(1)

    # KROK 2: Scrapuj artykuły
    all_results = []
    errors = []

    print("="*60)
    print("KROK 2: Scraping artykułów")
    print("="*60 + "\n")

    max_workers = 10
    print(f"Używam {max_workers} równoległych wątków\n")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(scrape_article, article_links), total=len(article_links)))

    # KROK 3: Zapisz wyniki
    timestamp = datetime.today().date()

    print(f"\n{'='*60}")
    print("KROK 3: Zapisywanie wyników")
    print("="*60)

    # JSON
    json_file = f'contekstualni_{timestamp}.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)
    print(f"  ✓ {json_file}")

    # Excel
    excel_file = f"contekstualni_{timestamp}.xlsx"
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(excel_file,
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Articles', index=False)
    print(f"  ✓ {excel_file}")

    # RAPORT KOŃCOWY
    print(f"\n{'='*60}")
    print("RAPORT KOŃCOWY")
    print("="*60)
    print(f"Pobranych artykułów: {len(all_results)}")
    print(f"Błędów: {len(errors)}")

    if errors and len(errors) <= 10:
        print(f"\nLinki z błędami:")
        for error_link in errors:
            print(f"  - {error_link}")
    elif errors:
        print(f"\nLinki z błędami (pierwsze 10):")
        for error_link in errors[:10]:
            print(f"  - {error_link}")
        print(f"  ... i {len(errors) - 10} więcej")

    # Statystyki kategorii
    if all_results:
        print(f"\n{'='*60}")
        print("STATYSTYKI KATEGORII")
        print("="*60)
        category_counts = {}
        for result in all_results:
            cat = result['Kategoria']
            category_counts[cat] = category_counts.get(cat, 0) + 1

        for cat, count in sorted(category_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"  {cat}: {count} artykułów")

    print(f"\n{'='*60}")
    print("GOTOWE!")
    print("="*60 + "\n")


SCRAPER CONTEKSTUALNI.PL
Blog literacko-kulturalny

KROK 1: Pobieranie artykułów z kategorii


Kategoria: Literatura
----------------------------------------
  ✓ Znaleziono 4 artykułów

Kategoria: Felietony
----------------------------------------
  ✓ Znaleziono 0 artykułów

Kategoria: Recenzje
----------------------------------------
  ✓ Znaleziono 8 artykułów

Kategoria: Rozmowy
----------------------------------------
  ✓ Znaleziono 3 artykułów

Kategoria: Sztuka
----------------------------------------
  ✓ Znaleziono 8 artykułów

Pobieranie ze strony głównej
  ✓ Znaleziono 20 artykułów

Łącznie znaleziono: 26 unikalnych artykułów

KROK 2: Scraping artykułów

Używam 10 równoległych wątków



100%|██████████| 26/26 [00:04<00:00,  5.42it/s]


KROK 3: Zapisywanie wyników
  ✓ contekstualni_2026-01-30.json
  ✓ contekstualni_2026-01-30.xlsx

RAPORT KOŃCOWY
Pobranych artykułów: 26
Błędów: 0

STATYSTYKI KATEGORII
  recenzje: 13 artykułów
  sztuka: 6 artykułów
  literatura: 4 artykułów
  rozmowy: 3 artykułów

GOTOWE!




/tmp/ipython-input-876800293.py:375: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Articles', index=False)


In [3]:
df.head()

,Link,Data publikacji,Kategoria,Tytuł artykułu,Autor,Tekst artykułu,Tagi,Linki zewnętrzne,Zdjęcia/Grafika,Filmy,Linki do zdjęć
0,https://contekstualni.pl/debiut/,no date,literatura,Debiut,,"Była końcówka listopada. Dzień miał być zimny,...","contekstualni, literatura, opowiadanie, paulin...",None,True,False,https://contekstualni.pl/wp-content/uploads/20...
1,https://contekstualni.pl/heavy-blood/,no date,literatura,Heavy Blood,,"Tak czy inaczej, bywałem również w ciotowatym ...","contekstualni, opowiadanie, paulina dąbkowska,...",None,True,False,https://contekstualni.pl/wp-content/uploads/20...
2,https://contekstualni.pl/make-room-for-oneself/,no date,recenzje,Make room for oneself,,"Teatr Łaźnia Nowa, Virginia Woolf,Własny pokój...","contekstualni, dramat, esej, łaźnianowa, pauli...",None,True,False,https://contekstualni.pl/wp-content/uploads/20...
3,https://contekstualni.pl/boza-misja-na-wyspie/,no date,recenzje,Boża misja na wyspie,,"Rozłożyło mnie, więc leżę w łóżku, faszeruję s...","BLOG LITERACKI, contekstualni, film, godland, ...",None,True,False,https://contekstualni.pl/wp-content/uploads/20...
4,https://contekstualni.pl/zabawa-w-udawanie/,no date,recenzje,Zabawa w udawanie,,W programie dwudziestego sezonu artystycznego ...,"latający potwór spaghetti, mateusz pakuła, pau...",None,True,False,https://contekstualni.pl/wp-content/uploads/20...
